In [1]:
import genjax
import jax

key = jax.random.PRNGKey(314159)

console = genjax.pretty()

In [2]:
@genjax.prox.unnorm.lang
@genjax.gen
def model():
    x = genjax.Normal(3.0, 1.0) @ "x"
    genjax.prox.score(-(x**3))
    return x

In [3]:
model

UnnormalizedMeasureFunction
└── gen_fn
    └── BuiltinGenerativeFunction
        └── source
            └── PytreeClosure
                ├── callable
                │   └── <function model>
                └── environment
                    └── list

In [8]:
target = genjax.prox.Target(model, (), genjax.EmptyChoiceMap())
key, tr = genjax.prox.Importance(500, None).simulate(key, (target,))
tr.get_retval()["x"]

Array(0.669816, dtype=float32)